# Ensemble Building

In [ ]:
import sys
sys.path.append('../')
from src.ensemble import EnsembleBuilder
import numpy as np 
import pandas as pd
import joblib 
import matplotlib.pyplot as plt
import seaborn as sns

# Load preprocessed data
X_train = np.load('../data/processed/X_train_processed.npy')
X_val = np.load('../data/processed/X_val_processed.npy')
y_train = np.load('../data/processed/y_train.npy')
y_val = np.load('../data/processed/y_val.npy')

#Load trained models
base_models = {}
model_names = ['decision_tree', 'random_forest', 'xgboost']

for name in model_names:
    base_models[name] = joblib.load(f'../data/models/{name}_best.joblib')


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/y_train_processed.npy'